# Run Notebooks on a schedule

In [ ]:
import warnings
warnings.filterwarnings("ignore") # ignore auth warnings

In [ ]:
from ghapi.all import GhApi
api = GhApi()

In [ ]:
nebari_events = api.list_events_parallel(owner='nebari-dev', repo='nebari', n_pages=1)

In [ ]:
for i in range(3):
    print(f"Member: {nebari_events[i]['actor']['login']} \nEvent: {nebari_events[i]["type"]} \nTime: {nebari_events[i]['created_at']} \n")

In [ ]:
with open('nebari_events.txt', 'a') as f:
    f.write(str(nebari_events))